In [11]:
from collections import Counter
import math

# Функция для построения дерева Шеннона-Фано
def build_tree(symbols, codes, prefix=""):
    if len(symbols) == 1:
        # Базовый случай: если символ только один, назначаем ему код
        symbol = symbols[0][0]
        codes[symbol] = prefix
        return

    # Разделяем символы на две группы с примерно одинаковыми вероятностями
    total_weight = sum([item[1] for item in symbols])
    running_sum = 0
    split_point = 0
    for i, (symbol, weight) in enumerate(symbols):
        running_sum += weight
        if running_sum >= total_weight / 2:
            split_point = i + 1
            break

    # Рекурсивно строим коды для левой и правой групп
    build_tree(symbols[:split_point], codes, prefix + "0")
    build_tree(symbols[split_point:], codes, prefix + "1")

# Функция для кодирования сообщения
def shannon_fano_encode(data):
    # Подсчет частоты появления символов
    frequency = Counter(data)

    # Сортировка символов по частоте
    sorted_symbols = sorted(frequency.items(), key=lambda x: x[1], reverse=True)

    # Построение кодов Шеннона-Фано
    codes = {}
    build_tree(sorted_symbols, codes)

    # Кодирование сообщения
    encoded_data = ''.join([codes[symbol] for symbol in data])

    return codes, encoded_data, frequency

# Функция для декодирования сообщения
def shannon_fano_decode(encoded_data, codes):
    # Инвертируем словарь кодов
    reverse_codes = {v: k for k, v in codes.items()}

    # Декодируем сообщение
    decoded_data = ""
    buffer = ""
    for bit in encoded_data:
        buffer += bit
        if buffer in reverse_codes:
            decoded_data += reverse_codes[buffer]
            buffer = ""

    return decoded_data

# Функция для вычисления энтропии
def calculate_entropy(frequency):
    total = sum(frequency.values())
    entropy = 0
    for count in frequency.values():
        probability = count / total
        entropy -= probability * math.log2(probability)
    return entropy

# Функция для вычисления средней длины кодирования (Lcp)
def calculate_average_code_length(codes, frequency):
    total_symbols = sum(frequency.values())
    avg_length = 0
    for symbol, code in codes.items():
        avg_length += len(code) * frequency[symbol]
    return avg_length / total_symbols

# Функция для построения бинарного дерева кодов
def build_binary_tree(codes):
    tree = {}
    for symbol, code in codes.items():
        current_node = tree
        for bit in code:
            if bit not in current_node:
                current_node[bit] = {}
            current_node = current_node[bit]
        current_node["symbol"] = symbol
    return tree

# Функция для вывода бинарного дерева
def print_binary_tree(tree, level=0):
    for key, value in tree.items():
        if key == "symbol":
            print("  " * level + f"Symbol: {value}")
        else:
            print("  " * level + f"Bit: {key}")
            print_binary_tree(value, level + 1)

# Пример использования
if __name__ == "__main__":
    data = "hello world"  # Можно заменить на любую строку чисел
    
    # Кодирование
    codes, encoded_data, frequency = shannon_fano_encode(data)
    print("Закодированный алфавит (в битах):", codes)
    print("Закодированные данные:", encoded_data)

    # Декодирование
    decoded_data = shannon_fano_decode(encoded_data, codes)
    print("Декодированные данные:", decoded_data)

    # Средняя длина кодирования
    avg_length = calculate_average_code_length(codes, frequency)
    print(f"Средняя длина кодирования (Lcp): {avg_length:.4f} бит")

    # Построение и вывод бинарного дерева
    binary_tree = build_binary_tree(codes)
    print("Бинарное дерево закодированных элементов:")
    print_binary_tree(binary_tree)

    # Предел энтропии для источника
    entropy = calculate_entropy(frequency)
    print(f"Предел энтропии для источника: {entropy:.4f} бит")


Закодированный алфавит (в битах): {'l': '00', 'o': '010', 'h': '011', 'e': '1000', ' ': '1001', 'w': '101', 'r': '110', 'd': '111'}
Закодированные данные: 01110000000010100110101011000111
Декодированные данные: hello world
Средняя длина кодирования (Lcp): 2.9091 бит
Бинарное дерево закодированных элементов:
Bit: 0
  Bit: 0
    Symbol: l
  Bit: 1
    Bit: 0
      Symbol: o
    Bit: 1
      Symbol: h
Bit: 1
  Bit: 0
    Bit: 0
      Bit: 0
        Symbol: e
      Bit: 1
        Symbol:  
    Bit: 1
      Symbol: w
  Bit: 1
    Bit: 0
      Symbol: r
    Bit: 1
      Symbol: d
Предел энтропии для источника: 2.8454 бит
